# **ADHUS/FAUHS Teacher-Staff Educational Research**
---------
## **Data Preparation for Power BI Dashboard**

Last Updated: THindle 2024 March 04

Created by: Tucker Hindle (thindle2016@fau.edu) - 2022 May 26

Google Sheet Key = `1QJvSUr-kQMqNv-lKt8RPDXXgxIAOERalqdw1NkgXeBs`


---------

# Overview

**Goal: Tidy `Teacher-Staff Edu Research Database` by re-structuring it to facilitate the creation of a Power BI dashboard.**

Four types of observational units (i.e., tables) will be created: `STUDIES`, `ENGAGEMENT`, `PUBLICATIONS`, and `PRESENTATIONS`.  Each will contain the following variables (i.e., columns):
*   Name of ADHUS/FAUHS Teacher-Staff Members
*   Year of study, publication, presentation, etc.

As well as additional variables relevant to each...

`STUDIES`: Title of Study, Subject Area, and Participant Group (including the specific grade level)

`ENGAGEMENT`: Type of engagement ("RCP" or "Teacher as Researcher" course)

`PUBLICATIONS`: Citation and Link/DOI

`PRESENTATIONS`: Conference and Title of Presentation

Currently, individual cells contain aggregated information - for example, multiple collaborators on a single study, several presentations given as a result of a particular study, and studies completed as a requirement for the "Teacher as Researcher" course.  Transformation of existing data will be required to handle such cases so that each observation forms a row and each cell contains a single value.

# Imports and Google Drive

[Code snippet](https://colab.research.google.com/notebooks/snippets/sheets.ipynb) for importing data using `gspread`, a Python API for Google Sheets ([Documentation](https://docs.gspread.org/en/latest/))

In [1]:
# imports
import pandas as pd

# Google Account authentication
from google.colab import auth
auth.authenticate_user()

# authorize access for Google Sheets
from google.auth import default
creds, _ = default()

import gspread
gc = gspread.authorize(creds)

# Read Google Sheets file into DataFrame

In [2]:
# open Google Sheets Workbook by key
wb = gc.open_by_key("1QJvSUr-kQMqNv-lKt8RPDXXgxIAOERalqdw1NkgXeBs")

# read first worksheet into DataFrame
sh = wb.sheet1
df = pd.DataFrame(sh.get_all_records())

df.head()

,Study Year,Study Title,ADHUS/FAUHS Educator Researchers,Additional Researchers,Grade Level of Participants,Subject Area of Study,Presented at,Presentation Title,Publication,Publication Year,DOI or Link,CERP Engagement
0,2010-2013,Budding Readers (Reading Buddies),Gina Sands,"Janet Towell, Lydia Bartram","PK, 6",Reading,,,,,,
1,2011-2012,Teaching conceptual knowledge about arithmetic...,Agnes Timar,Donald Ploger,"8, 9",Math,2014 Society for Information Technology and Te...,Mathematically Meaningful Mistakes; Robotic Ar...,,,,
2,2012-2013,Teachers’ use of instructional strategies foll...,"Chris Barzey, Jennifer O'Sullivan, Gina Hufty,...",Peggy Schooling,"2, 3, 5, 7, 8, 9",Math,,,,,,
3,2016-2017,The effect of digital game based learning on a...,"Jennifer O'Sullivan, Kelli Wilder, Caitlin Gut...","Tricia Meredith, Allan Phipps",4,STEM,,,,,,
4,2016-2017,Students' goals in a High School Research Meth...,Tricia Meredith,Maria Vasquez-Colina,9,Research,,,,,,


## Handle Case of Multiple Collaborators on a Single Study

In [3]:
# add column to represent each group as a list of collaborator names
df["Name"] = [group.split(", ") for group in df["ADHUS/FAUHS Educator Researchers"]]

# transform each list-like group to replicated rows, of which contains a single name
df = df.explode("Name")

df.head()

,Study Year,Study Title,ADHUS/FAUHS Educator Researchers,Additional Researchers,Grade Level of Participants,Subject Area of Study,Presented at,Presentation Title,Publication,Publication Year,DOI or Link,CERP Engagement,Name
0,2010-2013,Budding Readers (Reading Buddies),Gina Sands,"Janet Towell, Lydia Bartram","PK, 6",Reading,,,,,,,Gina Sands
1,2011-2012,Teaching conceptual knowledge about arithmetic...,Agnes Timar,Donald Ploger,"8, 9",Math,2014 Society for Information Technology and Te...,Mathematically Meaningful Mistakes; Robotic Ar...,,,,,Agnes Timar
2,2012-2013,Teachers’ use of instructional strategies foll...,"Chris Barzey, Jennifer O'Sullivan, Gina Hufty,...",Peggy Schooling,"2, 3, 5, 7, 8, 9",Math,,,,,,,Chris Barzey
2,2012-2013,Teachers’ use of instructional strategies foll...,"Chris Barzey, Jennifer O'Sullivan, Gina Hufty,...",Peggy Schooling,"2, 3, 5, 7, 8, 9",Math,,,,,,,Jennifer O'Sullivan
2,2012-2013,Teachers’ use of instructional strategies foll...,"Chris Barzey, Jennifer O'Sullivan, Gina Hufty,...",Peggy Schooling,"2, 3, 5, 7, 8, 9",Math,,,,,,,Gina Hufty


# General Purpose Function to Update Google Sheets

Avoid copy/paste for repeating the same code (perhaps with minor changes) in a single script. Rather, write general purpose functions to put code together.

This function should accept a DataFrame, Google Sheet (workbook), and worksheet name to be updated.  The existing worksheet will be removed and replaced with a blank worksheet, which will then be populated with the data provided in the DataFrame.

In [4]:
def update_gsheet(data, workbook, sheet_name):

    workbook.del_worksheet(workbook.worksheet(sheet_name)) # remove existing

    # create empty worksheet
    sheet = workbook.add_worksheet(title=sheet_name,
                                   rows=data.shape[0],
                                   cols=data.shape[1])

    # populate with most recent data
    sheet.update([data.columns.values.tolist()] + data.values.tolist())

# **Studies**

Educational Research by ADHUS/FAUHS Faculty & Staff

In [5]:
# select columns relevant to studies
studies = df[
    [
        "Name",
        "Study Year",
        "Study Title",
        "Grade Level of Participants",
        "Subject Area of Study"
    ]
]

studies.tail()

,Name,Study Year,Study Title,Grade Level of Participants,Subject Area of Study
85,Katherine Hendrickson,2022-2023,,,
86,Brenna McBrinn,2022-2023,,,
87,Marijayne McMahon,2022-2023,,,
88,,,,,
89,,,,,


In [6]:
# add column to represent study participants as a list of school grades
studies["Grade"] = [str(row).split(", ") for row in studies["Grade Level of Participants"]]

# transform each list-like to replicated rows, of which contains a single grade
studies = studies.explode("Grade")

studies.tail()

<ipython-input-6-f3807cc2c719>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  studies["Grade"] = [str(row).split(", ") for row in studies["Grade Level of Participants"]]


,Name,Study Year,Study Title,Grade Level of Participants,Subject Area of Study,Grade
85,Katherine Hendrickson,2022-2023,,,,
86,Brenna McBrinn,2022-2023,,,,
87,Marijayne McMahon,2022-2023,,,,
88,,,,,,
89,,,,,,


In [7]:
# define a function that will categorize students according to grade level
def participant_groups(grade):

    el = ["PK", "K", "1", "2", "3", "4", "5"]
    ms = ["6", "7", "8"]
    hs = ["9", "10", "11", "12"]
    uni = "University"

    if grade in el:
        return "Elementary"
    elif grade in ms:
        return "Middle School"
    elif grade in hs:
        return "High School"
    elif grade == uni:
        return "University"
    elif grade == "":
        return ""
    else:
        return "Non-Student"

# apply this function to a column and store new 'student category' variable
studies["Participant Group"] = studies["Grade"].apply(participant_groups)

In [8]:
# keep relevant columns only
studies = studies.drop(columns="Grade Level of Participants")

studies.tail()

,Name,Study Year,Study Title,Subject Area of Study,Grade,Participant Group
85,Katherine Hendrickson,2022-2023,,,,
86,Brenna McBrinn,2022-2023,,,,
87,Marijayne McMahon,2022-2023,,,,
88,,,,,,
89,,,,,,


In [9]:
# update Google Sheet
update_gsheet(studies, wb, "STUDIES")

# **Comprehensive Educator Research Program Engagement**

ADHUS/FAUHS Faculty & Staff in "RCP" or "Teacher as Researcher" course

In [10]:
# select subset that participated in the program
engagement = df.loc[df["CERP Engagement"] != ""]

# handle case of 'RCP' and 'Teacher as Researcher' completed in the same year
engagement["Engagement"] = [row.split(", ") for row in engagement["CERP Engagement"]]
engagement = engagement.explode("Engagement")

# select columns relevant to program engagement
engagement = engagement[
    [
        "Name",
        "Study Year",
        "Engagement"
    ]
]

engagement = engagement.rename(columns={"Study Year": "Academic Year"})

engagement.head()

<ipython-input-10-c2aa56a250ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engagement["Engagement"] = [row.split(", ") for row in engagement["CERP Engagement"]]


,Name,Academic Year,Engagement
6,Agnes Timar,2018-2019,Teacher as Researcher course
7,Amy Tift,2018-2019,RCP
8,Caterina Siciliano,2018-2019,RCP
9,Cheri Celesti,2018-2019,Teacher as Researcher course
10,Christopher Clevenger,2018-2019,RCP


In [11]:
# update Google Sheet
update_gsheet(engagement, wb, "ENGAGEMENT")

# **Publications**

Educational Research by ADHUS/FAUHS Faculty & Staff

In [12]:
# select subset that have a publication
publications = df.loc[df["Publication"] != ""]

# keep columns relevant to publications
publications = publications[
    [
        "Name",
        "Publication Year",
        "Publication",
        "DOI or Link"
    ]
]

print(f'Total Publications: {len(publications["Publication"].unique())}\n')
print(publications["Publication"].unique())

Total Publications: 15

['Meredith, T.L. and Chamely-Wiik, D.C. 2017. Ensuring the safety and success of dual enrolled students involved in undergraduate research through a university-high school partnership. Council on Undergraduate Research Quarterly. 37(4): 41-42.'
 'Vaughan, M., Cavallaro, C., Baker, J., Celesti, C., Clevenger, C., Darling, H., Kasten, R., Laing, M., Marbach, R., Timar, A., & Wilder, K. (2019). Positioning teachers as researchers: Lessons in empowerment, change, and growth. Florida Journal of Educational Research, 57(2), 133-139.'
 'Baker, J. (2019). The impact of choice novels on student reading motivation. Florida Journal of Educational Research, 57(2), 62-71'
 'Ingle, D., Bruellman, R., Espana, E., Galloway, K., Anderson, T., Meredith, T.L., Porter, M.E. 2019. Science Behind the Lesson: It’s a bird! It’s a plane! It’s biomechanics! CourseSource.'
 'Ingle, D., Bruellman, R., Espana, E., Galloway, K., Anderson, T., Meredith, T.L., Porter, M.E. 2019. Lesson: It’s a

In [13]:
# update Google Sheet
update_gsheet(publications, wb, "PUBLICATIONS")

# **Presentations**

ADHUS/FAUHS Faculty & Staff at Conferences, Symposia, etc.

In [14]:
# select subset that have given a presentation
presentations = df.loc[df["Presented at"] != ""]

# keep relevant columns only
presentations = presentations[
    [
        "Name",
        "Presented at",
        "Presentation Title"
    ]
]

presentations.head()

,Name,Presented at,Presentation Title
1,Agnes Timar,2014 Society for Information Technology and Te...,Mathematically Meaningful Mistakes; Robotic Ar...
6,Agnes Timar,2018 Florida Educational Research Association ...,Positioning teachers as researchers: Lessons i...
7,Amy Tift,2019 SAC Symposium,Going Old-School: Rejection of 1:1 iPad Note-T...
8,Caterina Siciliano,2019 SAC Symposium,Social & Emotional Learning Intervention in th...
9,Cheri Celesti,2018 Florida Educational Research Association ...,Positioning teachers as researchers: Lessons i...


## Pair Conferences/Symposia with corresponding Presentation Titles

### Get multiple presentations as separate columns (wide format)

#### Conferences, symposia, etc.

In [15]:
# split locations (delimited by semicolons) into separate columns
conferences = presentations["Presented at"].str.split(pat="; ", expand=True)

# get column labels with meaningful prefix
conference_labels = [f"Conference_{col}" for col in list(conferences.columns)]

presentations[conference_labels] = conferences # add columns

#### Presentation titles

In [16]:
# split titles (delimited by semicolons) into separate columns
titles = presentations["Presentation Title"].str.split(pat="; ", expand=True)

# get column labels with meaningful prefix
title_labels = [f"Presentation_{col}" for col in list(titles.columns)]

presentations[title_labels] = titles # add columns

#### Pair together

In [17]:
# get (conference, presentation title) pairs formatted as a wide dataset
event_labels = []
i = 0
for pair in zip(conference_labels, title_labels):
    presentations[f"Event_{i}"] = presentations[pair[0]] + "; " + presentations[pair[1]]
    event_labels.append(f"Event_{i}")
    i += 1

presentations.head()

,Name,Presented at,Presentation Title,Conference_0,Conference_1,Conference_2,Presentation_0,Presentation_1,Presentation_2,Event_0,Event_1,Event_2
1,Agnes Timar,2014 Society for Information Technology and Te...,Mathematically Meaningful Mistakes; Robotic Ar...,2014 Society for Information Technology and Te...,2012 Florida Conference on Recent Advances in ...,2012 American Society for Engineering Educatio...,Mathematically Meaningful Mistakes,Robotic Art for STEM,Exploring Engineering Technology Practice to E...,2014 Society for Information Technology and Te...,2012 Florida Conference on Recent Advances in ...,2012 American Society for Engineering Educatio...
6,Agnes Timar,2018 Florida Educational Research Association ...,Positioning teachers as researchers: Lessons i...,2018 Florida Educational Research Association ...,None,None,Positioning teachers as researchers: Lessons i...,None,None,2018 Florida Educational Research Association ...,NaN,NaN
7,Amy Tift,2019 SAC Symposium,Going Old-School: Rejection of 1:1 iPad Note-T...,2019 SAC Symposium,None,None,Going Old-School: Rejection of 1:1 iPad Note-T...,None,None,2019 SAC Symposium; Going Old-School: Rejectio...,NaN,NaN
8,Caterina Siciliano,2019 SAC Symposium,Social & Emotional Learning Intervention in th...,2019 SAC Symposium,None,None,Social & Emotional Learning Intervention in th...,None,None,2019 SAC Symposium; Social & Emotional Learnin...,NaN,NaN
9,Cheri Celesti,2018 Florida Educational Research Association ...,Positioning teachers as researchers: Lessons i...,2018 Florida Educational Research Association ...,None,None,Positioning teachers as researchers: Lessons i...,None,None,2018 Florida Educational Research Association ...,NaN,NaN


## Tidy Presentations Dataset

Tidying this dataset requires first melting (into long format), and then splitting the `Conference Presentation` column into three variables: `Year`, `Conference`, and `Presentation`.

* `melt()` is used to unpivot a DataFrame from wide to long format

* String splitting is used to get each variable into a separate column


In [18]:
# transform dataset by unpivoting columns into rows
presentations = presentations.melt(id_vars=["Name"],
                                   value_vars=event_labels,
                                   value_name="Conference Presentation")

presentations = presentations.dropna() # remove row if no presentation

presentations.head()

,Name,variable,Conference Presentation
0,Agnes Timar,Event_0,2014 Society for Information Technology and Te...
1,Agnes Timar,Event_0,2018 Florida Educational Research Association ...
2,Amy Tift,Event_0,2019 SAC Symposium; Going Old-School: Rejectio...
3,Caterina Siciliano,Event_0,2019 SAC Symposium; Social & Emotional Learnin...
4,Cheri Celesti,Event_0,2018 Florida Educational Research Association ...


In [19]:
# parse out the year into separate column
presentations[["Presentation Year", "intermediate"]] = presentations["Conference Presentation"].str.split(pat=" ", n=1, expand=True)

# split again into two variables, Conference and Presentation Title
presentations[["Conference", "Presentation Title"]] = presentations["intermediate"].str.split(pat="; ", expand=True)

# keep relevant columns only
presentations = presentations[
    [
        "Name",
        "Presentation Year",
        "Conference",
        "Presentation Title"
    ]
]

presentations.head()

,Name,Presentation Year,Conference,Presentation Title
0,Agnes Timar,2014,Society for Information Technology and Teacher...,Mathematically Meaningful Mistakes
1,Agnes Timar,2018,Florida Educational Research Association (FERA),Positioning teachers as researchers: Lessons i...
2,Amy Tift,2019,SAC Symposium,Going Old-School: Rejection of 1:1 iPad Note-T...
3,Caterina Siciliano,2019,SAC Symposium,Social & Emotional Learning Intervention in th...
4,Cheri Celesti,2018,Florida Educational Research Association (FERA),Positioning teachers as researchers: Lessons i...


In [20]:
# update Google Sheet
update_gsheet(presentations, wb, "PRESENTATIONS")